In [1]:
from IPython.display import display, Markdown, Latex
import scipy.stats
import scipy.optimize
import numpy as np

#%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt

* MOSFET used: DMN2041L
* GPIO voltage: $3.0 V$ ($V_{dd} - 0.3 V$). All values in the datasheet assume $V_{GS}$ of $2.5 V$ or $4.5 V$ so we use $2.5 V$.
* Temperature: $25 °C$ (room temperature)
* Imax: $500 mA$

## From datasheet

* $R_{ds(on)max} = 41 m\Omega @ 2.5 V$
* Thermal Resistance, Junction to Ambient @ TA=+25 °C: $R_{\theta JA}=161 °C/W$

In [2]:
# From manual measurements of "Fig. 5 On-Resistance Variation with Temperature"
measurements=[
    [-50, 0.75],
    [0, 1],
    [50, 1.1],
    [150, 1.57],
]

#slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(measurements)

def rdson_factor_for_temp(temp):
    x=rdson_factor_for_temp.x
    return x.intercept + (temp*x.slope)
rdson_factor_for_temp.x = scipy.stats.linregress(measurements)

display(Markdown("Slope and intercept for $R_{{DS(on)}}$ vs $T$:\nSlope = ${:0.5f}$, Intercept = ${:0.5f}$".format(
    rdson_factor_for_temp.x.slope, rdson_factor_for_temp.x.intercept)))

#x = np.linspace(-50, 150, 100)
#plt.plot(x, rdson_factor_for_temp(x))
#plt.title("$R_{DS(on)}$ vs Temperature")
#plt.show()

Slope and intercept for $R_{DS(on)}$ vs $T$:
Slope = $0.00399$, Intercept = $0.95543$

In [3]:
Von=2.5
Ta=25
Imax=500e-3
Imax=2
Rdson=0.041
TR=161

def calc_temp(I):
    def mosfet_temp(I):
        # Uses the global Rdson and Ta
        def mosfet_temp_single(T, Rdson, I):
            R=Rdson*rdson_factor_for_temp(T)
            P=I*I*R

            Trise=TR*P
            T=Ta+Trise

            return T

        return scipy.optimize.fixed_point(mosfet_temp_single, [Rdson, I], args=(Rdson, I), xtol=1e-3)

    res = mosfet_temp(I)
    if res[0] == res[1]:
        msg = "Temperature: ${:0.2f} ^{{\circ}}C$ at $I={:.0f} mA$ and $T_{{A}} = {} ^{{\circ}}C$".format(res[0], I * 1000, Ta)
    else:
        msg = "Could not converge, temp is between {} and {} ^{{\circ}}C".format(res[0], res[1])

    display(Markdown(msg))
    
calc_temp(0.1)
calc_temp(0.500)
calc_temp(1)
calc_temp(2)

Temperature: $25.07 ^{\circ}C$ at $I=100 mA$ and $T_{A} = 25 ^{\circ}C$

Temperature: $26.75 ^{\circ}C$ at $I=500 mA$ and $T_{A} = 25 ^{\circ}C$

Temperature: $32.15 ^{\circ}C$ at $I=1000 mA$ and $T_{A} = 25 ^{\circ}C$

Temperature: $56.14 ^{\circ}C$ at $I=2000 mA$ and $T_{A} = 25 ^{\circ}C$

In [4]:
def f(I):
    t=mosfet_temp(I)
    return t[0] if t[0] == t[1] else (t[0] + t[1])/2 # None

i = np.linspace(0.001, 5, 100) # 6.4 A is 
#plt.plot(i, np.vectorize(f, otypes=[float])(i))
#plt.title("$I$ vs Temperature")
#plt.xlabel("Current")
#plt.ylabel("Temperature")
#plt.grid()
#plt.show()